# TEST 1: Leveraging LLMs for Feature Generation and Classification

Typically, if our data has $N$ features, we need around $10N$ data items to reach peak performance with classic classifiers like Logistic Regression. Therefore, if our vocabulary has 10,000 words, we would need around 1 million items in the training set to reach peak performance.

An interesting idea regarding this was explored in 2024 in [Balek, V., S'ykora, L., Sklen'ak, V., & Kliegr, T. (2024). LLM-based feature generation from text for interpretable machine learning. ArXiv, abs/2409.07132](https://arxiv.org/abs/2409.07132). The idea is to use an LLM to generate meaningful and interpretable features from text, and then use Logistic Regression for classification.

For example, in the movie plots dataset, we could have features like:
- "Is the protagonist an animal?" (0 or 1)
- "Does the plot indicate psychological suffering?" (0 or 1)

With a reasonable number of these features, our model could make predictions based on meaningful features instead of raw words.

## Objectives
* Perform feature extraction for a particular dataset
* Compare performance and explainability of classifiers with different approaches. 

## Rules

I highlight a few elements of our usual rules:

* You are **NOT ALLOWED** to use AI to generate any code you are asked to make yourself. This includes ChatGPT, CoPilot and all similar generators.
* You are **NOT ALLOWED** to use Google or any other search engine.
* You are **ALLOWED** to use the offical documentations for libraries: 
    * [sklearn](https://scikit-learn.org/)
    * [numpy](https://numpy.org/)
    * [matplotlib](https://matplotlib.org/)
    * [google AI studio](https://aistudio.google.com/)
* You are **ALLOWED** to use previous code from this course as basis.
* You **MUST** use the university's proctoring software to show you are complying with these rules
* This task is **INDIVIDUAL**. DO NOT share your code or results with anyone else.

## Tasks and Deliverables

* At any point, refer to [Balek et al.](https://arxiv.org/abs/2409.07132). 
* Make a well-commented code to solve each one of the tasks below.
* Each task will be evaluated as:
    * Insufficient: task is not done, off-topic, or low-effort
    * In process: task is incomplete, done with a clear conceptual error, or comments 
    * Proficient: everything works and comments are enough to understand what is being done
    * Advanced: everything works, comments are enough to understand what is being done, and code is well organized and formated using functions, dataclasses, and other adequate structures.
* This task should be finished by the end of the class.
* After you are finished, submit the executed notebook in our LMS system.

### 1. Dataset Preparation:
Adapting Balek et al.'s strategy to our movie plot classification case, create a dataset with at least 100 labeled items and at least 5 meaningful features. None of the features can be the class itself ("is this a drama plot?"). Use a clear strategy to avoid exceeding free tier quotas. Store data locally in a format of your choice.

### 2. Classification:
Use the generated features to train a Logistic Regression model. Use cross-validation to select the best hyperparameters. Report accuracy and f1-score for your classifier.

### 3. Performance Comparison
Compare the performance of the following approaches:
1. Traditional Bag-of-Words
2. LLM-generated features with Logistic Regression
3. Direct classification using LLM

Use a bar plot to show the performance differences (choose either accuracy or F1-score).

### 4. Improvement Strategies
Determine whether labeling more items would improve system performance. Use data to justify your answer.


In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
import google.generativeai as genai
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [109]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [110]:
df = pd.read_csv('https://raw.githubusercontent.com/tiagoft/NLP/main/wiki_movie_plots_drama_comedy.csv')
df.head()

,Plot,Genre
0,The film is about a family who move to the sub...,comedy
1,Before heading out to a baseball game at a nea...,comedy
2,The plot is that of a black woman going to the...,comedy
3,On a beautiful summer day a father and mother ...,drama
4,A thug accosts a girl as she leaves her workpl...,drama


## Generating meaningful features and interpable feature from text - using LLM for feature generation

In [111]:
df_5 = df.head(5)

prompt = f"""

Using this dataset of movies plots and theirs genres, we want to have the most 5 effective questions that can segregate the genres of the movies based on the plot summaries.

The dataset:
{df_5}

Give me only the questions that can segregate the genres of the movies based on the plot summaries.

The answears must be in a list format.
"""

generation_config = genai.GenerationConfig(
    max_output_tokens=500,
    temperature=0.0,
)

model = genai.GenerativeModel(model_name="gemini-1.5-flash")

response = model.generate_content(prompt,
                                  generation_config=generation_config)

print(response.text)

KeyboardInterrupt: 

In [112]:
list_questions = response.text.split('\n')[1]
list_questions = ['Does the plot primarily focus on humorous situations and characters?', 'Are the central conflicts and themes related to serious life issues or emotional struggles?', 'Does the plot involve a significant amount of physical violence or criminal activity?', 'Is the overall tone of the plot lighthearted and comedic, or serious and dramatic?', 'Does the plot revolve around a clear-cut, easily resolved conflict, or are the issues more complex and ambiguous?']
list_questions

['Does the plot primarily focus on humorous situations and characters?',
 'Are the central conflicts and themes related to serious life issues or emotional struggles?',
 'Does the plot involve a significant amount of physical violence or criminal activity?',
 'Is the overall tone of the plot lighthearted and comedic, or serious and dramatic?',
 'Does the plot revolve around a clear-cut, easily resolved conflict, or are the issues more complex and ambiguous?']

In [113]:
for question in list_questions:
    df[question] = np.nan

df.head()

,Plot,Genre,Does the plot primarily focus on humorous situations and characters?,Are the central conflicts and themes related to serious life issues or emotional struggles?,Does the plot involve a significant amount of physical violence or criminal activity?,"Is the overall tone of the plot lighthearted and comedic, or serious and dramatic?","Does the plot revolve around a clear-cut, easily resolved conflict, or are the issues more complex and ambiguous?"
0,The film is about a family who move to the sub...,comedy,NaN,NaN,NaN,NaN,NaN
1,Before heading out to a baseball game at a nea...,comedy,NaN,NaN,NaN,NaN,NaN
2,The plot is that of a black woman going to the...,comedy,NaN,NaN,NaN,NaN,NaN
3,On a beautiful summer day a father and mother ...,drama,NaN,NaN,NaN,NaN,NaN
4,A thug accosts a girl as she leaves her workpl...,drama,NaN,NaN,NaN,NaN,NaN


In [114]:
def classify_content(row):
    prompt = f"""
    I have a text and 5 questions. I would like to use the text to answer the questions.
    The output should be just a list that contains the boolean values like: [1, 0, 1, 0, 1]
    Text: {row}
    Questions: {list_questions}
    """
    response = model.generate_content(prompt,
                                      generation_config=generation_config)
    return response.text


In [115]:
from tqdm import tqdm
import time
df = df.head(50)

In [116]:
for i, row in tqdm(df.iterrows(), total=len(df), desc='Classifying content'):
    row_text = row['Plot']
    answear = list(classify_content(row_text))
    answear = [int(i) for i in answear if i.isdigit()]
    df.loc[i, list_questions] = answear
    time.sleep(3)
df.head()

Classifying content:   0%|          | 0/50 [04:41<?, ?it/s]


KeyboardInterrupt: 